# Лабораторная работа №7
## Синтезатор частот с дробным делителем
### Звягин М.

Основу данной лабораторной работы составляет ФАПЧ, реализованный в ЛР6. Однако, из-за того, что ФАПЧ был реализован для гармонических сигналов и делитель частоты реализован не отдельным блоком, а операцией деления мгновенной фазы с ГУН, сначала необходимо модифицировать ФАПЧ, чтобы он работал с цифровыми сигналами (прямоульными импульсами). Также необходимо отдельно реализовать блок делителя частоты

Добавление используемых библиотек

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

## Фазовый детектор
Данная реализация предназначена для работы с цифровыми сигналами

In [ ]:
def Gen_Pulse(input_signal):
    if input_signal > 0:
        pulse = 1
    else:
        pulse = 0
    return pulse

class PhaseFrequencyDetector:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.up = 0
        self.down = 0
        self.prev_ref = 0
        self.prev_fb = 0
    
    def update(self, ref_signal, fb_signal):

        ref_rising = (ref_signal == 1) and (self.prev_ref == 0)
        fb_rising = (fb_signal == 1) and (self.prev_fb == 0)
        
        if ref_rising and not fb_rising:
            self.up = 1
        elif fb_rising and not ref_rising:
            self.down = 1
        elif ref_rising and fb_rising:
            self.up = 0
            self.down = 0
        
        if self.up and self.down:
            self.up = 0
            self.down = 0
        
        self.prev_ref = ref_signal
        self.prev_fb = fb_signal
        
        return self.up, self.down


class ChargePump:
    def __init__(self, current_gain):
        self.current_gain = current_gain
    
    def process(self, up, down):
        return self.current_gain * (up - down)

In [ ]:
fs = 1e6
f = 1e3
t = np.arange(0, 5*fs/f)/fs
signal = np.sin(2 * np.pi * f * t)

pulse = []
for sample in signal:
    pulse.append((sample > 0).astype(float))
    
plt.figure(figsize=(10, 5))
plt.plot(t, signal, label='Исходный гармонический сигнал')
plt.plot(t, pulse, label='Преобразованный цифровой сигнал', linestyle='--')
plt.xlabel('Время, с')
plt.ylabel('Амплитуда')
plt.title('Преобразование гармонического сигнала в цифровой')

In [ ]:
pd = PhaseFrequencyDetector()
cp = ChargePump(1)

fs = 1e6
f = 1e3
t = np.arange(0, 5*fs/f)/fs
factor_f_vco_list = [0.5, 2]

for factor_f_vco in factor_f_vco_list:
    s_in = np.sin(2 * np.pi * f * t)
    s_vco = np.sin(2 * np.pi * factor_f_vco*f * t + 0*np.pi/4)

    s_in_pulse = np.zeros(np.shape(s_in))
    s_vco_pulse = np.zeros(np.shape(s_vco))
    s_phase_detector = np.zeros(np.shape(s_in))
    s_phase_detector_up = np.zeros(np.shape(s_in))
    s_phase_detector_down = np.zeros(np.shape(s_in))
    s_charge_pump = np.zeros(np.shape(s_in))

    for i in np.arange(len(s_in)):
        s_in_pulse[i] = Gen_Pulse(s_in[i])
        s_vco_pulse[i] = Gen_Pulse(s_vco[i])
        s_phase_detector_up[i], s_phase_detector_down[i] = pd.update(s_in_pulse[i], s_vco_pulse[i])
        s_charge_pump[i] = cp.process(s_phase_detector_up[i], s_phase_detector_down[i])

    plt.figure(figsize=(10, 5))
    plt.subplot(3, 1, 1)
    plt.plot(t, s_in_pulse, label='Внешний входной сигнал')
    plt.plot(t, s_vco_pulse, label='Сигнал с ГУН', linestyle='--')  
    plt.xlabel('Время, с')
    plt.ylabel('Амплитуда')
    plt.title('Сигналы на входе фазового детектора')
    plt.legend()

    plt.subplot(3, 1, 2)    
    plt.plot(t, s_charge_pump, label='Выход цифрового фазового детектора', linestyle=':')
    plt.xlabel('Время, с')
    plt.ylabel('Амплитуда')
    plt.title('Выход цифрового фазового детектора')
    plt.legend()
    plt.tight_layout()


Как видно по полученному графику в отличие от фазового детектора для гармонических сигналов на выходе цифрового фазового детектора сигнал также является цифровым (последовательностью импульсов).

Первый рисунок показывает результат работы фазового детектора, когда частота внешнего входного сигнала меньше частоты ГУН. В этом случае среднее значение выхода фазового детектора > 0.

Второй рисунок показывает результат работы фазового детектора, когда частота внешнего входного сигнала больше частоты ГУН. В этом случае среднее значение выхода фазового детектора < 0.

## Петлевой фильтр
Класс петлевого фильтр взят из ЛР6 без изменений, так как подходит и для цифрового ФАПЧ

In [ ]:
class Loop_Filter:
    # Конструктор класса
    def __init__(self, fs, K_pd, K_nco):
        self.integral_part = 0
        self.damping_factor = np.sqrt(2)/2
        self.natural_freq = 2*np.pi*fs/100
        self.equivalent_noise_bandwidth = self.natural_freq/2*(self.damping_factor + 1/(4*self.damping_factor))
        
        self.K_p = 1/(K_pd*K_nco)*(4*self.damping_factor/(self.damping_factor + 1/(4*self.damping_factor)))*self.equivalent_noise_bandwidth/fs
        self.K_i = 1/(K_pd*K_nco)*(4/(self.damping_factor + 1/(4*self.damping_factor))**2)*(self.equivalent_noise_bandwidth/fs)**2
    
    # Функция для обновления полосы попускания
    def update_equivalent_noise_bandwidth(self):
        self.equivalent_noise_bandwidth = self.natural_freq/2*(self.damping_factor + 1/(4*self.damping_factor))
        
    # Функция для обновления параметров
    def update_loop_factors(self, fs, K_pd, K_nco):        
        self.K_p = 1/(K_pd*K_nco)*(4*self.damping_factor/(self.damping_factor + 1/(4*self.damping_factor)))*self.equivalent_noise_bandwidth/fs
        self.K_i = 1/(K_pd*K_nco)*(4/(self.damping_factor + 1/(4*self.damping_factor))**2)*(self.equivalent_noise_bandwidth/fs)**2
        
    # Функции для установки параметров
    def set_damping_factor(self, damping_factor):
        self.damping_factor = damping_factor
        
    def set_natural_freq(self, natural_freq):
        self.natural_freq = natural_freq

    def set_equivalent_noise_bandwidth(self, equivalent_noise_bandwidth):
        self.equivalent_noise_bandwidth = equivalent_noise_bandwidth
        
    # Функции для получения параметров
    def get_damping_factor(self):
        return self.damping_factor
    
    def get_natural_freq(self):
        return self.natural_freq  
    
    def get_equivalent_noise_bandwidth(self):
        return self.equivalent_noise_bandwidth      
    
    def get_K_p(self):    
        return self.K_p
    
    def get_K_i(self):
        return self.K_i

    # Основная функция вызова
    def __call__(self, error_sample):
        proportional_part = self.K_p*error_sample
        self.integral_part += self.K_i*error_sample
        return proportional_part + self.integral_part, self.integral_part

## Генератор, управляемый напряжением
Класс ГУН также взят без изменений, однако после получения с его выхода комплексного гармонического сигнала для корректной работы цифрового ФАПЧ необходимо преобразовывать его в цифровой сигнал (последовательность импульсов)

In [ ]:
class NCO:
    def __init__(self):
        self.phase = 0
        self.delta_phase_NCO = 0
        self.K_nco = 1
    
    def set_freq(self, freq_nco, fs):
        self.delta_phase_NCO = 2*np.pi*freq_nco/fs            
    
    def set_factor(self, factor):
        self.K_nco = factor
              
    def get_freq(self):
        return self.delta_phase_NCO

    def get_factor(self):
        return self.K_nco
    
    def update_phase(self, delta_phase_LP):
        self.phase += self.K_nco*delta_phase_LP + self.delta_phase_NCO
        
    def __call__(self, delta_phase_LP):
        
        self.update_phase(delta_phase_LP)
        return np.exp(1j*self.phase), self.phase

In [ ]:
fs = 1e6
Ts = 1/fs
f = 1e3

t = np.arange(0, 5*fs/f)/fs
s = 0.1*(-0.8 + np.sin(2*np.pi*f*t))


nco = NCO()
nco.set_freq(1000, fs)
nco.set_factor(0.1)

s_res = np.zeros(shape=np.shape(s), dtype=complex)
s_phase = np.zeros(shape=np.shape(s))
for i in np.arange(0, len(s)):
    s_res[i], s_phase[i] = nco(s[i])

plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(t*1e3, s)
plt.title('Управляющий сигнал NCO')
plt.ylabel('Амплитуда')
plt.xlabel('Время, t, мс')
plt.grid()
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(t*1e3, (np.real(s_res) > 0).astype(float), label='Цифровой выход NCO')
plt.plot(t*1e3, np.real(s_res), label='Гармонический выход NCO', linestyle='--')
plt.title('Выходной сигнал NCO')
plt.ylabel('Амплитуда')
plt.xlabel('Время, t, мс')
plt.grid()
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(t*1e3, s_phase)
plt.title('Мгновенная фаза NCO')
plt.ylabel('Значение, рад')
plt.xlabel('Время, t, мс')
plt.grid()
plt.legend()
plt.tight_layout()

## Цифровой ФАПЧ

In [ ]:
def Digital_PLL(fs, f_vco, f_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, visible):
    
    pd = PhaseFrequencyDetector()
    cp = ChargePump(1)

    LP = Loop_Filter(fs, K_pd=K_pd, K_nco=K_nco)
    LP.set_damping_factor(demp_factor)
    LP.set_natural_freq(2*np.pi*fs/freq_nat)
    LP.set_equivalent_noise_bandwidth(factor_bw*fs)
    # LP.update_equivalent_noise_bandwidth()
    LP.update_loop_factors(fs, K_pd, K_nco)
 
    nco = NCO()
    nco.set_freq(f_vco, fs)
    nco.set_factor(K_nco)

    t = np.arange(0, number_periods*fs/f_sig)/fs
    signal_input = (np.sin(2*np.pi*f_sig*t) > 0).astype(float)

    phase_detector_out = np.zeros_like(signal_input)
    loop_filter_out = np.zeros_like(signal_input)
    nco_out = np.zeros_like(signal_input).astype(complex)
    nco_phase = np.zeros_like(signal_input)
    signal_out = np.zeros_like(signal_input)
    signal_error = np.zeros_like(signal_input)
    s_phase_detector_up = np.zeros_like(signal_input)
    s_phase_detector_down = np.zeros_like(signal_input)

    for i in np.arange(1, len(signal_input)):
        nco_out[i], nco_phase[i] = nco(loop_filter_out[i-1])
        signal_out[i] = (np.imag(nco_out[i]) > 0).astype(float)
        signal_error[i] = signal_input[i] - signal_out[i]
        nco_out[i] = (np.imag(nco_out[i]) > 0).astype(float)
        
        s_phase_detector_up[i], s_phase_detector_down[i] = pd.update(signal_input[i], nco_out[i])
        phase_detector_out[i] = cp.process(s_phase_detector_up[i], s_phase_detector_down[i])

        loop_filter_out[i], _ = LP(phase_detector_out[i])

    if visible:
        plt.figure(figsize=(12, 8))
        
        plt.subplot(5, 1, 1)
        plt.plot(t, signal_input, label='Входной сигнал')
        plt.plot(t, signal_out, label='Выход NCO', linestyle='--')
        plt.title('Входной и выходной сигналы')
        plt.ylabel('Уровень')
        plt.grid()
        plt.legend()
        
        plt.subplot(5, 1, 2)
        plt.plot(t, phase_detector_out)
        plt.title('Выход фазового детектора')
        plt.ylabel('Ошибка')
        plt.grid()
        
        plt.subplot(5, 1, 3)
        plt.plot(t, loop_filter_out)
        plt.title('Выход петлевого фильтра')
        plt.ylabel('Управляющий сигнал')
        plt.grid()
        
        plt.subplot(5, 1, 4)
        plt.plot(t, nco_phase/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(5, 1, 5)
        plt.plot(t, signal_error)
        plt.title('Ошибка ФАПЧ')
        plt.xlabel('Время, с')
        plt.ylabel('Значение')
        plt.grid()
        
        plt.tight_layout()
        
    return signal_input, signal_out, phase_detector_out, loop_filter_out, nco_out, signal_error, t, nco_phase

In [ ]:
fs = 1e6
f_vco = 1e3
f_sig = 1.5e3
K_pd = 1
K_nco = 1
freq_nat = 1
demp_factor = 0.707
factor_bw = 0.001
number_periods = 100
visible = True

signal_input, signal_out, _, _, _, _, _, _ = Digital_PLL(fs, f_vco, f_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, visible)

Представленный результат показывает корректную работу цифровго ФАПЧ, однако он обладает следующими недостатками:
1) В установившемся режиме существует достаточно маленькая разность фаз между входным и локально сгенерированным сигналом
2) Из предыдущего пункта вытекает наличие "периодической ошибки" на графике ошибки ФАПЧ, что скорее всего должно привести к мешающему пику на спектре

In [ ]:
def Get_Spectrum(signal, fs):
    N = len(signal)
    signal_fft = np.fft.fft(signal)
    signal_fft = np.abs(signal_fft[:N//2])/N
    freq = np.fft.fftfreq(N, 1/fs)[:N//2]
    return freq, signal_fft

In [ ]:
freq_1, signal_fft_1 = Get_Spectrum(signal_input, fs)
freq_2, signal_fft_2 = Get_Spectrum(signal_out, fs)


plt.figure(figsize=(10, 5))
plt.plot(freq_1, 10*np.log10(signal_fft_1), label='Входной внешний сигнал')
plt.plot(freq_2, 10*np.log10(signal_fft_2), '--', label='Выходной локальный сигнал')

plt.title('Амплитудный спектр')
plt.xlabel('Частота, Гц')
plt.ylabel('Уровень, дБ')
plt.grid()
plt.legend()
plt.ylim(-35, 0)
plt.xlim(0, 1e5)


В целом сравнение спекторов входного и выходного сигнала показало, что в спектре выходного сигнала "общий шум" выше, чем у исходного. Это связано с начальным временем установления, а также постоянным небольшим расхождением фаз 

## Синтезатор частот с целочисленным делителем
Модифицируем версию из ЛР6 путем добавления цифрового ФАПЧ и отдельного блока делителя частоты

In [ ]:
class FrequencyDivider:
    def __init__(self, divider_ratio):

        self.division_ratio = divider_ratio
        self.counter = 0
        self.last_output = None
        self.last_input = None
    
    def __call__(self, input_sample):
        if self.last_output is None:
            self.last_output = 1 - input_sample
            self.last_input = 1- input_sample
            return self.last_output
            
        
        trigger_edge = ((input_sample == 1) and (self.last_input == 0)) or ((input_sample == 0) and (self.last_input == 1))
        
        if trigger_edge:
            self.counter += 1
            
            if self.counter >= self.division_ratio:
                self.last_output = 1 - self.last_output
                self.counter = 0
        
        self.last_input = input_sample
        return self.last_output
    
    def reset(self):
        self.counter = 0
        self.last_output = None
        self.last_input = None

In [ ]:
f = 1e3
t = np.arange(0, 10*fs/f)/fs
signal_input = (np.sin(2*np.pi*f_sig*t) > 0).astype(float)

plt.figure(figsize=(10, 10))
divider_list = [1, 2, 3, 4, 5]
for index, divider in enumerate(divider_list): 
    Divider = FrequencyDivider(divider)
    output_signal = []
    for sample in signal_input:
        output_signal.append(Divider(sample))
        
    Divider.reset()
    plt.subplot(5, 1, index+1)
    plt.plot(t, signal_input, label='Входной сигнал')
    plt.plot(t, output_signal, '--', label=f'Выходной сигнал (деление на {Divider.division_ratio})')
    plt.title('Целочисленный делитель частоты')
    plt.xlabel('Время, с')
    plt.ylabel('Амплитуда')
    plt.legend()
    plt.grid()
    plt.tight_layout()


Полученные результаты доказывают корректность работы целочисленного делителя частоты

In [ ]:
def Natural_Freq_Synt(M, N, fs, f_vco, f_sig, p_sig,  K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, visible):

    pd = PhaseFrequencyDetector()
    cp = ChargePump(1)
    
    divider_M = FrequencyDivider(M)
    divider_N = FrequencyDivider(N)
    
    LP = Loop_Filter(fs, K_pd=K_pd, K_nco=K_nco)
    LP.set_damping_factor(demp_factor)
    LP.set_natural_freq(2*np.pi*fs/freq_nat)
    LP.set_equivalent_noise_bandwidth(factor_bw*fs)
    LP.update_equivalent_noise_bandwidth()
    LP.update_loop_factors(fs, K_pd, K_nco)
 
    nco = NCO()
    nco.set_freq(f_vco, fs)
    nco.set_factor(K_nco)

    t = np.arange(0, number_periods*fs/f_sig)/fs
    signal_input = (np.sin(2*np.pi*f_sig*t + p_sig) > 0).astype(float)
    for i, sample in enumerate(signal_input):
        signal_input[i] = divider_M(sample)

    phase_detector_out = np.zeros((np.shape(signal_input)), dtype=complex)
    loop_filter_out = np.zeros((np.shape(signal_input)))

    nco_out = np.zeros((np.shape(signal_input)), dtype=complex)
    nco_out_result = np.zeros((np.shape(signal_input)), dtype=complex)
    nco_phase = np.zeros((np.shape(signal_input)))

    signal_out = np.zeros((np.shape(signal_input)))
    s_phase_detector_up = np.zeros((np.shape(signal_input)))
    s_phase_detector_down = np.zeros((np.shape(signal_input)))

    for i in np.arange(1, len(signal_input)):
        nco_out_result[i], nco_phase[i] = nco(loop_filter_out[i-1])
        signal_out[i] = (np.imag(nco_out_result[i]) > 0).astype(float)
        
        nco_out[i] = (np.real(nco_out_result[i]) > 0).astype(float)
        nco_out[i] = divider_N(nco_out[i])
        
        s_phase_detector_up[i], s_phase_detector_down[i] = pd.update(signal_input[i], nco_out[i])
        phase_detector_out[i] = cp.process(s_phase_detector_up[i], s_phase_detector_down[i])
        loop_filter_out[i], _ = LP(phase_detector_out[i])

    divider_N.reset()
    divider_M.reset()   
    
    if visible:
        plt.figure(figsize=(10,10))
        plt.subplot(5, 1, 1)
        plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
        plt.title(f'Выход фазового детектора. M = {M}. N = {N}')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 2)
        plt.plot(np.arange(0, len(loop_filter_out))/fs*1e6, loop_filter_out, '-', label = 'Выход петлевого фильтра')
        plt.title('Выход петлевого фильтра')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 4)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '--', label = 'Входной сигнал')
        plt.title('Сравнение входного и выходного сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 3)
        plt.plot(np.arange(0, len(nco_phase))/fs*1e6, nco_phase/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(5, 1, 5)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        sig_target = (np.sin(2*np.pi*(f_sig*N/M)*t + p_sig) > 0).astype(float)
        plt.plot(np.arange(0, len(sig_target))/fs*1e6, sig_target, '--', label = 'Ожидаемый сигнал')
        plt.xlim((len(sig_target) - 0.2*len(sig_target)), len(sig_target))
        plt.title('Сравнение выходного и идеального требуемого сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        

    return t, signal_input, signal_out

In [ ]:
fs = 1e5
f_vco = 1.05e2
f_sig = 10
p_sig = 0
K_pd = 1
K_nco = 1
freq_nat = 5000
demp_factor = 1
factor_bw = 1

number_periods = 150
visible = True
M = 1
N = 10

t, sig_in, sig_out = Natural_Freq_Synt(M, N, fs, f_vco, f_sig, p_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, True)

По полученным результатам видно, что синтезатор частот с целочисленным делителем работает, но не очень точно, так как видна небольшая разница частот между выходным сигналом и идеальным требуемым сигналом. В приведенном выше моделировании в установившемся состоянии разница частот составляет примерно 0,25 Гц, из-за чего и уплывает выходной сигнал относительно идеального.

In [ ]:
fs = 1e5
f_vco = 1.05e2
f_sig = 10
p_sig = 0
K_pd = 1
K_nco = 1
freq_nat = 5000
demp_factor = 1
factor_bw = 1

number_periods = 150
visible = True
M = 1
N = [10, 15, 20, 25, 30, 35, 40]

for n in N:
    t, sig_in, sig_out = Natural_Freq_Synt(M, n, fs, f_vco, f_sig, p_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, True)

По результатам видно, что при заданных параметрах синтезатор частоты с целочисленным делителем работает корректно (но с наличием постоянной частотной ошибки) при заданных N = 10, ..., 40. 

По графикам "Частота ГУН" можно увидеть, что для всех моделирований частота приближается к требуемой 

## Синтезатор частот с дробным делителем
Необходимо изменить делитель частоты, чтобы была возможность получать нецелочисленные коэффициенты деления.

В качестве делителя частоты с дробным коэффициентом будет использована простая реализация, в которой будет происходить деление на целочисленные коэффиенты с разной частотой (в зависимости от установленного дробного коэффициента деления). Например, для коэффициента деления 3.5 будет осуществляться поочередное деление на 3 и 4.

In [ ]:
class FractionalDivider:
    def __init__(self, divider_ratio):

        self.divider = divider_ratio
        self.acc = 0.0       
        self.counter = 0       
        self.output = None          
        self.last_input = None    
        self.set_next_divider()
        
    def set_next_divider(self):
        self.acc += self.divider - int(self.divider)
        if self.acc >= 1.0:
            self.acc -= 1.0
            self.next_divider = int(self.divider) + 1
        else:
            self.next_divider = int(self.divider)

    def __call__(self, input_sample):
        if self.output is None:
            self.output = 1 - input_sample
            self.last_input = 1 - input_sample
            return self.output
        
        if (input_sample == 1 and self.last_input == 0) or (input_sample == 0 and self.last_input == 1):
            self.counter += 1

            if self.counter >= self.next_divider:
                self.output = 1 - self.output
                self.counter = 0
                self.set_next_divider()  
                
        self.last_input = input_sample
        return self.output
    
    def reset(self):
        self.acc = 0.0
        self.counter = 0
        self.output = None
        self.last_input = None
        self.next_divider = 0

In [ ]:
fs = 1e5
f_sig = 1e3
t = np.arange(0, 50*fs/f_sig)/fs

N = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3]

input_signal = (np.sin(2*np.pi*f_sig*t) > 0).astype(float) 

plt.figure(figsize=(10, 30))
for index, n in enumerate(N):
    divider = FractionalDivider(n)
    output_signal = [divider(sample) for sample in input_signal]
    div_signal = (np.sin(2*np.pi*(f_sig/n)*t) > 0).astype(float)
    divider.reset()
    
    plt.subplot(len(N), 2, 2*index+1)
    plt.plot(t, div_signal, label='Требуемый сигнал')
    plt.plot(t, output_signal, '--', label=f'Выходной сигнал (деление на {divider.divider})')
    plt.title('Дробный делитель частоты')
    plt.xlabel('Время, с')
    plt.ylabel('Амплитуда')
    plt.legend()
    plt.grid()
    
    freq_1, signal_fft_1 = Get_Spectrum(div_signal, fs)
    freq_2, signal_fft_2 = Get_Spectrum(output_signal, fs)
    plt.subplot(len(N), 2, 2*index+2)
    plt.plot(freq_1, 10*np.log10(signal_fft_1), label='Входной внешний сигнал')
    plt.plot(freq_2, 10*np.log10(signal_fft_2), '--', label='Выходной локальный сигнал')
    plt.title('Амплитудный спектр')
    plt.xlabel('Частота, Гц')
    plt.ylabel('Уровень, дБ')
    plt.grid()
    plt.legend()
    plt.ylim(-35, 0)
    plt.xlim(0, 4e4)
    plt.tight_layout()


По полученным результатам видно, что используемый дробный делитель частоты работает, однако данный вид делителя имеет недостатки. Из-за того, что целочисленные коэффициенты деления не перемешиваются, а идут друг за другом (например, 2 2 2 2 2 3 2 2 2 2 3..), в выходном сигнале заметны моменты перехода к другому коэффиценту деления, что увеличивает джиттер выходного сигнала.

На графиках спектра видно, что основная составляющая выходного сигнала после дробного делителя всегда совпадает с основной составляющей требуемого сигнала. Однако при чем дальше дробная часть от 0.5, тем больше мешающих составляющих рядом с основной частотой.

In [ ]:
class FractionalDivider_Dither:
    def __init__(self, divider_ratio):

        self.divider = divider_ratio
        self.acc = 0.0    
        self.counter = 0       
        self.output = None          
        self.last_input = None    
        self.dither_amplitude = 0.5
        self.set_next_divider()
        
    def set_next_divider(self):
        noise = np.random.uniform(-self.dither_amplitude, self.dither_amplitude)
        self.acc += self.divider - int(self.divider)
        if self.acc + noise >= 1.0:
            self.acc -= 1.0
            self.next_divider = int(self.divider) + 1
        else:
            self.next_divider = int(self.divider)

    def __call__(self, input_sample):
        if self.output is None:
            self.output = 1 - input_sample
            self.last_input = 1 - input_sample
            return self.output
        
        if (input_sample == 1 and self.last_input == 0) or (input_sample == 0 and self.last_input == 1):
            self.counter += 1

            if self.counter >= self.next_divider:
                self.output = 1 - self.output
                self.counter = 0
                self.set_next_divider()  
                
        self.last_input = input_sample
        return self.output
    
    def reset(self):
        self.acc = 0.0
        self.counter = 0
        self.output = None
        self.last_input = None
        self.next_divider = 0

In [ ]:
fs = 1e5
f_sig = 1e3
t = np.arange(0, 50*fs/f_sig)/fs

N = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3]

input_signal = (np.sin(2*np.pi*f_sig*t) > 0).astype(float) 

plt.figure(figsize=(10, 30))
for index, n in enumerate(N):
    divider = FractionalDivider_Dither(n)
    output_signal = [divider(sample) for sample in input_signal]
    div_signal = (np.sin(2*np.pi*(f_sig/n)*t) > 0).astype(float)
    divider.reset()
    
    plt.subplot(len(N), 2, 2*index+1)
    plt.plot(t, div_signal, label='Требуемый сигнал')
    plt.plot(t, output_signal, '--', label=f'Выходной сигнал (деление на {divider.divider})')
    plt.title('Дробный делитель частоты')
    plt.xlabel('Время, с')
    plt.ylabel('Амплитуда')
    plt.legend()
    plt.grid()
    
    freq_1, signal_fft_1 = Get_Spectrum(div_signal, fs)
    freq_2, signal_fft_2 = Get_Spectrum(output_signal, fs)
    plt.subplot(len(N), 2, 2*index+2)
    plt.plot(freq_1, 10*np.log10(signal_fft_1), label='Входной внешний сигнал')
    plt.plot(freq_2, 10*np.log10(signal_fft_2), '--', label='Выходной локальный сигнал')
    plt.title('Амплитудный спектр')
    plt.xlabel('Частота, Гц')
    plt.ylabel('Уровень, дБ')
    plt.grid()
    plt.legend()
    plt.ylim(-35, 0)
    plt.xlim(0, 4e4)
    plt.tight_layout()


Выше представлены результаты для модифированного дробного делителя частоты. В реализованный выше делитель был добавлен дизер (шум), который должен был сделать выбор целочисленных коэффициентов более хаотичным, что должно было частично устранить периодическое изменение фаз в выходном графике. 

Такая реализация лишь немного позволила убрать периодическую ошибку, но в целом визуально ситуация практически не поменялась (на спектрах все еще видно достаточно высокие мешающие частоты).

Добавим реализованный дробный делитель частоты в реализацию синтезатора частот

In [ ]:
def Fractional_Freq_Synt(M, N, fs, f_vco, f_sig, p_sig,  K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, visible):

    pd = PhaseFrequencyDetector()
    cp = ChargePump(1)
    
    divider_M = FractionalDivider_Dither(M)
    divider_N = FractionalDivider_Dither(N)
    
    LP = Loop_Filter(fs, K_pd=K_pd, K_nco=K_nco)
    LP.set_damping_factor(demp_factor)
    LP.set_natural_freq(2*np.pi*fs/freq_nat)
    LP.set_equivalent_noise_bandwidth(factor_bw*fs)
    LP.update_equivalent_noise_bandwidth()
    LP.update_loop_factors(fs, K_pd, K_nco)
 
    nco = NCO()
    nco.set_freq(f_vco, fs)
    nco.set_factor(K_nco)

    t = np.arange(0, number_periods*fs/f_sig)/fs
    signal_input = (np.sin(2*np.pi*f_sig*t + p_sig) > 0).astype(float)
    for i, sample in enumerate(signal_input):
        signal_input[i] = divider_M(sample)

    phase_detector_out = np.zeros((np.shape(signal_input)), dtype=complex)
    loop_filter_out = np.zeros((np.shape(signal_input)))

    nco_out = np.zeros((np.shape(signal_input)), dtype=complex)
    nco_out_result = np.zeros((np.shape(signal_input)), dtype=complex)
    nco_phase = np.zeros((np.shape(signal_input)))

    signal_out = np.zeros((np.shape(signal_input)))
    s_phase_detector_up = np.zeros((np.shape(signal_input)))
    s_phase_detector_down = np.zeros((np.shape(signal_input)))

    for i in np.arange(1, len(signal_input)):
        nco_out_result[i], nco_phase[i] = nco(loop_filter_out[i-1])
        signal_out[i] = (np.imag(nco_out_result[i]) > 0).astype(float)
        
        nco_out[i] = (np.real(nco_out_result[i]) > 0).astype(float)
        nco_out[i] = divider_N(nco_out[i])
        
        s_phase_detector_up[i], s_phase_detector_down[i] = pd.update(signal_input[i], nco_out[i])
        phase_detector_out[i] = cp.process(s_phase_detector_up[i], s_phase_detector_down[i])
        loop_filter_out[i], _ = LP(phase_detector_out[i])

    divider_N.reset()
    divider_M.reset()   
    
    if visible:
        plt.figure(figsize=(10,10))
        plt.subplot(5, 1, 1)
        plt.plot(np.arange(0, len(phase_detector_out))/fs*1e6, phase_detector_out, '-', label = 'Выход фазового детектора')
        plt.title(f'Выход фазового детектора. M = {M}. N = {N}')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 2)
        plt.plot(np.arange(0, len(loop_filter_out))/fs*1e6, loop_filter_out, '-', label = 'Выход петлевого фильтра')
        plt.title('Выход петлевого фильтра')
        plt.xlabel('Время t, мкс ')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 4)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        plt.plot(np.arange(0, len(signal_input))/fs*1e6, signal_input, '--', label = 'Входной сигнал')
        plt.title('Сравнение входного и выходного сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()

        plt.subplot(5, 1, 3)
        plt.plot(np.arange(0, len(nco_phase))/fs*1e6, nco_phase/(2*np.pi*t), '-', label = 'Частота ГУН')
        plt.title('Частота ГУН')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        
        plt.subplot(5, 1, 5)
        plt.plot(np.arange(0, len(signal_out))/fs*1e6, signal_out, '-', label = 'Выходной сигнал')
        sig_target = (np.sin(2*np.pi*(f_sig*N/M)*t + p_sig) > 0).astype(float)
        plt.plot(np.arange(0, len(sig_target))/fs*1e6, sig_target, '--', label = 'Ожидаемый сигнал')
        plt.xlim((len(sig_target) - 0.2*len(sig_target)), len(sig_target))
        plt.title('Сравнение выходного и идеального требуемого сигналов')
        plt.xlabel('Время t, мкс ')
        plt.grid()
        plt.legend()
        plt.tight_layout()
        

    return t, signal_input, signal_out

In [ ]:
fs = 1e5
f_vco = 1.05e2
f_sig = 10
p_sig = 0
K_pd = 1
K_nco = 1
freq_nat = 5000
demp_factor = 1
factor_bw = 1

number_periods = 150
visible = True
M = 1
N = 20.5

t, sig_in, sig_out = Fractional_Freq_Synt(M, N, fs, f_vco, f_sig, p_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, True)

Полученные результаты в целом соответствуют результатам, полученным при моделировании реализованного синтезатора частот с целочисленным коэффициентом деления, а именно система подстраивает частоту ГУН к требуемой, но с определенной ошибкой. Например, для моделирования выше ошибка составляет 0,6 Гц. (При использовании дробного коэффициента 20,5 частоту ГУН должна быть 205 Гц, но установилась в состояние 205,6 Гц).

В целом можно считать, что синтезатор частот с дробным делителем работает корректно (если не нужна точная реализация).

Рассмотрим выход синтезатора частот при коэффициентах деления от 20,1 до 21. Вместе с этим определим ошибку установившейся частоты в каждом из случаев.

In [ ]:
fs = 1e5
f_vco = 1.05e2
f_sig = 10
p_sig = 0
K_pd = 1
K_nco = 1
freq_nat = 5000
demp_factor = 1
factor_bw = 1

number_periods = 150
visible = True
M = 1
N = 20 + np.arange(0, 11)/10

for n in N:
    t, sig_in, sig_out = Fractional_Freq_Synt(M, n, fs, f_vco, f_sig, p_sig, K_pd, K_nco, freq_nat, demp_factor, factor_bw, number_periods, True)

Полученные результаты подтвержают выводы, полученные выше.
Рассмотрим ошибки частоты для каждого из выбранных коэффициентов деления:
1) N = 20.0
Ошибка частоты составляет 0,6
2) N = 20.1
Ошибка частоты составляет 0,55
3) N = 20.2
Ошибка частоты составляет 0,5
4) N = 20.3
Ошибка частоты составляет 0,6

Для следующих N до 30 ошибка составляет ~0.6

Таким образом, при выбранных параметрах моделирования ошибка составила 0,6 и не менялась при изменениии коэффициента деления.

## Вывод:

В данной лабораторной работе был реализован и исследован синтезатор частот с дробным делителем. Основой для реализации послужил ФАПЧ, реализованный в ЛР6. Однако в ЛР6 был реализован ФАПЧ и синтезатор частот с целочисленным делителем, где в качестве делителя использовалась математическая операция. Поэтому сначала был модифицирован ФАПЧ (а именно изменен его фазовый детектор для работы с цифровыми, а не гармоническими сигналами). Это было необходимо для того, чтобы реализовать делитель частоты отдельным блоком, который работает только с цифровыми сигналами.

### Фазовый детектор
Реализован для цифровых сигналов с помощью D-триггеров. Выходом фазового детектора являются 2 сигнала (up и down), которые определяют длительности задержек входных сигналов друг относительно друга. Далее эти сигналы идут на "зарядовый насос".

### Зарядовый насос
Сигналы up и dowm поступают на "зарядовый насос", который преобразует эти сигналы в общий сигнал ошибки. Его реализация представляет собой простую операцию: K*(up - down).

### Петлевой фильтр
Был взят без изменений из ЛР6

### Генератор, управляемый напряжением
Был взят без изменений из ЛР6

### Целочисленный делитель частоты
Реализован через счетчик, который переключает фазу цифрового сигнала при накоплении значения, установленного в качестве параметра целочисленного делителя.

Была проверена его работа. Результы получены корректные.

### Модификация синтезатора частот с целочисленным делителем
Был реализован синтезатор частот с целочисленным делителем, но уже с отдельным блоком делителя и новым фазовым детектором. Результаты моделирования показали его корректную работу, однако стоит отметить, что данная реализация имеет ошибку частоты в установишемся состоянии, из-за чего выходной сигнал плывет относительно идеального требуемого.

### Дробный делитель частоты
Для синтезатора частот с дробным делителем необходимо было реализовать дробный делитель. 
Для дробного делителя представлена реализация, при которой целочисленные коэффициенты выбираются поочередно в зависимости от выбранного значения дробного коэффициента деления. Например, при установленном значении 3,4 сигнал будет делиться на 3 3 3 4 4 3 3 3 4 4...

Для данного типа делителя заметны переходы коэффициента деления, что увеличивает джиттер сигнала. Для его уменьшения была попытка модифицировать делитель путем добавления в него дизера (шума). Визуально данная модификация не улучшила ситуацию (что также видно при сравнении спектров).

### Синтезатор частот с дробным коэффициентом деления
Вместо целочисленного делителя был установлен дробный делитель. Были проведены моделирования с различными коэффициентами деления. Результаты показали аналогичное поведение, что и для синтезатора с целочисленным делителем, а именно система работает корректно, но существует частотная ошибка после установления, из-за чего плывет выходной сигнал относительно идеального требуемого.
